In [1]:
import pandas as pd 
import numpy as np 
from scipy.optimize import root, LinearConstraint
import matplotlib.pyplot as plt 

from simplepower.models import GridModel, ORPDHandler, GridDataClass, FileType

In [7]:
grid_data = GridDataClass("simplepower//models//grid_model_data//IEEE 14-bus RAW.txt", FileType.IEEE, f_nom=50, S_base_mva=100)
grid_model = GridModel(grid_data) 

V_lim_min = [0.9, 0.9, 0.95, 0.95, 0.95, 0.95, 0.95, 0.95, 0.95, 0.95, 0.95, 0.95, 0.95, 0.95]
V_lim_max = [1.1, 1.1, 1.05, 1.05, 1.05, 1.05, 1.05, 1.05, 1.05, 1.05, 1.05, 1.05, 1.05, 1.05]

# opf_model = ORPDHandler(grid_model, [0, 1, 2, 3, 4], I_lim_mat=grid_data.I_lims_pu.to_numpy())
opf_model = ORPDHandler(grid_model, [0, 1, 2, 3, 4], V_min=V_lim_min, V_max=V_lim_max)
pf_sol = grid_model.powerflow()
pf_sol.get_sol_df()

,P_inj MW,Q_inj_Mvar,V_bus_pu,delta_bus_deg
0,2.324010e+02,-1.628808e+01,1.060000,0.000000
1,1.830000e+01,3.124411e+01,1.045000,-4.980866
2,-9.420000e+01,6.166090e+00,1.010000,-12.717189
3,-4.780000e+01,3.900000e+00,1.017525,-10.295506
4,-7.600000e+00,-1.600000e+00,1.018966,-8.778556
5,-1.120000e+01,3.543696e+00,1.070000,-13.659270
6,-1.005103e-07,-3.898126e-10,1.060940,-13.086670
7,-1.665335e-14,1.798234e+01,1.090000,-13.086670
8,-2.950000e+01,-1.660000e+01,1.055126,-14.599968
9,-9.000000e+00,-5.800000e+00,1.050268,-14.719552


In [8]:
pf_sol.get_P_losses()/100

0.13401042180612366

In [9]:
V_init = grid_data._grid_gens["v_set_pu"].values 
V_init 

array([1.06 , 1.045, 1.01 , 1.07 , 1.09 ])

In [10]:
opf_sol = opf_model.solve_ORPD()
opf_sol.x 

array([1.1       , 1.07826798, 1.04149032, 1.05      , 1.05      ])

In [11]:
opf_sol.fun

12.856575478586867

In [12]:
V_opt = opf_sol.x 
opf_model.set_opt_v(opf_sol) 
grid_model.powerflow().get_sol_df() 

,P_inj MW,Q_inj_Mvar,V_bus_pu,delta_bus_deg
0,2.318566e+02,7.061219e+00,1.100000,0.000000
1,1.830000e+01,3.832324e+01,1.078268,-4.549728
2,-9.420000e+01,1.300740e+01,1.041490,-11.866601
3,-4.780000e+01,3.900000e+00,1.034341,-9.327748
4,-7.600000e+00,-1.600000e+00,1.036567,-7.876242
5,-1.120000e+01,-1.830079e+01,1.050000,-12.662621
6,2.245371e-07,-4.051653e-08,1.049159,-12.154752
7,-2.866631e-14,5.011974e-01,1.050000,-12.154752
8,-2.950000e+01,-1.660000e+01,1.045089,-13.727775
9,-9.000000e+00,-5.800000e+00,1.038391,-13.838889


# 30-bus system

In [30]:
grid_data = GridDataClass("simplepower//models//grid_model_data//IEEE 30-bus RAW.txt", FileType.IEEE, f_nom=50, S_base_mva=100)
grid_model = GridModel(grid_data) 

opf_model = ORPDHandler(grid_model, [0, 1, 2, 3, 4, 5], V_min=0.9, V_max=1.1)
pf_sol = grid_model.powerflow()
print(pf_sol.get_P_losses())
V_init = pf_sol.V_buses[[0, 1, 4, 7, 10, 12]]
pf_sol.get_sol_df()

17.555571827029333


,P_inj MW,Q_inj_Mvar,V_bus_pu,delta_bus_deg
0,2.609556e+02,-2.023098e+01,1.060000,0.000000
1,1.830000e+01,4.352282e+01,1.045000,-5.375115
2,-2.400000e+00,-1.200000e+00,1.020894,-7.535253
3,-7.600000e+00,-1.600000e+00,1.011953,-9.287980
4,-9.420000e+01,1.664599e+01,1.010000,-14.136818
5,-2.391379e-08,-1.000359e-08,1.010652,-11.034173
6,-2.280000e+01,-1.090000e+01,1.002613,-12.835091
7,-3.000000e+01,6.035369e+00,1.010000,-11.775160
8,1.523479e-09,-2.561031e-08,1.051011,-13.810747
9,-5.800000e+00,-2.000000e+00,1.045404,-15.327868


In [31]:
opf_model.const(V_init) 

array([1.06      , 1.045     , 1.02089377, 1.01195332, 1.01      ,
       1.01065175, 1.00261296, 1.01      , 1.05101079, 1.04540429,
       1.082     , 1.05881268, 1.071     , 1.04389278, 1.03902034,
       1.04533402, 1.04043297, 1.029115  , 1.0263944 , 1.03035897,
       1.03309182, 1.03365003, 1.02828456, 1.02238203, 1.01829733,
       1.00063736, 1.0243112 , 1.00712863, 1.00449503, 0.99303356])

In [32]:
opf_sol = opf_model.solve_ORPD()
print(opf_sol.fun)
V_opt = opf_sol.x 
opf_model.set_opt_v(opf_sol) 
grid_model.powerflow().get_sol_df() 

16.156830521210424


,P_inj MW,Q_inj_Mvar,V_bus_pu,delta_bus_deg
0,2.595568e+02,-1.849241e+01,1.100000,0.000000
1,1.830000e+01,3.405597e+01,1.083890,-4.935566
2,-2.400000e+00,-1.200000e+00,1.062704,-6.965477
3,-7.600000e+00,-1.600000e+00,1.053910,-8.578637
4,-9.420000e+01,1.505389e+01,1.051370,-13.034503
5,-9.247546e-08,-1.438520e-07,1.054455,-10.218369
6,-2.280000e+01,-1.090000e+01,1.045848,-11.861345
7,-3.000000e+01,1.953156e+01,1.058491,-10.970951
8,2.974734e-08,-3.299833e-08,1.086026,-12.777593
9,-5.800000e+00,-2.000000e+00,1.083899,-14.184805


In [33]:
opf_sol

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 16.156830521210424
       x: [ 1.100e+00  1.084e+00  1.051e+00  1.058e+00  1.100e+00
            1.100e+00]
     nit: 9
     jac: [-2.936e+01 -1.725e-02 -1.103e-03 -2.310e-03 -1.727e+00
           -2.434e+00]
    nfev: 122
    njev: 9